In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)


with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, "\n")
    print(b, "\n")
    print(usage)

Here is a simple recipe for making soju at home:

Ingredients:
- 1.5 liters of water
- 1 cup of rice
- 1 tablespoon of yeast
- 1 cup of sugar

Instructions:
1. Rinse the rice thoroughly under cold water until the water runs clear.
2. In a large pot, bring the water to a boil and add the rinsed rice.
3. Reduce the heat to low and simmer the rice for about 30 minutes, or until it becomes soft and mushy.
4. Remove the pot from heat and let it cool down to room temperature.
5. Once the rice has cooled, transfer it to a large glass jar or container.
6. Dissolve the yeast in a small amount of warm water and add it to the rice.
7. Stir in the sugar until it is fully dissolved.
8. Cover the jar or container with a clean cloth or plastic wrap and secure it tightly.
9. Store the mixture in a cool, dark place for about 7-10 days, allowing it to ferment.
10. After the fermentation period, strain the mixture through a cheesecloth or fine mesh strainer to remove any solids.
11. Transfer the strained

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI

chat = OpenAI(
    temperature=0.1,
    max_tokens=450,
    model="gpt-3.5-turbo-16k"
)

chat.save("model.json")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

chat = load_llm("model.json")

chat
